# Atividade 1
## Item 3 - Problema MAX-QBF com Set Cover

In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import sys
import io

In [2]:
def solve_max_sc_qbf_linearized(input_stream):
    """
    Resolve o problema MAX-QBF com Set Cover (MAX-SC-QBF)
    utilizando a linearização da função objetivo quadrática.

    Args:
        input_stream: Um objeto de fluxo de entrada (ex: sys.stdin ou um arquivo aberto).
    """

    # 1. Leitura dos dados da instância conforme Item 3.1 da Atividade [4, 5]
    n = int(input_stream.readline().strip()) # Número de variáveis binárias / subconjuntos / elementos [4]

    # Ler a linha de contagens de elementos por subconjunto (não usado diretamente no modelo, mas parte do formato) [4]
    _ = input_stream.readline().strip().split()

    # Ler as listas de elementos cobertos por cada subconjunto [4]
    # subsets_covered_elements[i] conterá uma lista de elementos (0-indexados) cobertos pelo subconjunto i.
    subsets_covered_elements = []
    for _ in range(n):
        line_elements = list(map(int, input_stream.readline().strip().split()))
        # Ajustar elementos para 0-indexados, já que a entrada é 1-indexada (ex: {1,2} se torna {0,1}) [informação fora das fontes, mas comum em programação]
        subsets_covered_elements.append([e - 1 for e in line_elements])
    
    # Reconstruir a matriz A (assumindo formato triangular superior) [4]
    A = [[0.0] * n for _ in range(n)]
    # A entrada para a matriz A é triangular superior [4].
    # Ela é lida linha por linha, preenchendo a diagonal e acima.
    for i in range(n):
        # Ler os coeficientes da linha atual (do a_ii até a_in)
        line_coeffs = list(map(float, input_stream.readline().strip().split()))
        col_idx_in_line = 0
        for j in range(i, n): # j começa de i para preencher a diagonal e acima
            A[i][j] = line_coeffs[col_idx_in_line]
            col_idx_in_line += 1

    # 2. Criar um novo modelo Gurobi
    model = gp.Model("MAX_SC_QBF_Linearized")

    # 3. Adicionar as variáveis binárias originais (x_i)
    # x[i] representa a i-ésima variável binária, indicando a seleção do subconjunto (i+1) [2]
    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    # 4. Adicionar as variáveis de linearização (y_ij)
    # y[i, j] representa o produto x_i * x_j.
    # Como A é triangular superior, só precisamos de y_ij para i <= j.
    y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

    # 5. Definir a função objetivo linearizada
    # A função objetivo original do MAX-QBF é Z = ∑_{i=1 to n} ∑_{j=1 to n} a_ij * x_i * x_j [3]
    # Com a linearização y_ij = x_i * x_j, e considerando que A[i][j] é 0 para i > j (matriz triangular superior),
    # a função objetivo se torna linear: Z = ∑_{i=0 to n-1} ∑_{j=i to n-1} A[i][j] * y[i, j]
    obj = gp.quicksum(A[i][j] * y[i, j] for i in range(n) for j in range(n) if i <= j)
    model.setObjective(obj, GRB.MAXIMIZE)

    # 6. Adicionar as restrições de linearização
    # Para cada par (i, j) onde y_ij é definido (i <= j), adicionamos as três restrições para garantir y_ij = x_i * x_j.
    for i in range(n):
        for j in range(i, n): # Iterar sobre i <= j
            # Restrição 1: y_ij <= x_i
            model.addConstr(y[i, j] <= x[i], f"linear_y{i}_{j}_le_x{i}")
            # Restrição 2: y_ij <= x_j
            model.addConstr(y[i, j] <= x[j], f"linear_y{i}_{j}_le_x{j}")
            # Restrição 3: y_ij >= x[i] + x[j] - 1
            model.addConstr(y[i, j] >= x[i] + x[j] - 1, f"linear_y{i}_{j}_ge_sum_minus_1")
            
            # Para i == j, estas restrições forçam y[i,i] = x[i], o que é correto para x_i * x_i = x_i.

    # 7. Adicionar as restrições de cobertura de conjuntos (Set Cover) [2]
    # Para todo elemento k_element no universo N (de 0 a n-1 em nossa indexação interna),
    # deve existir ao menos um subconjunto S_i selecionado (x_i=1) tal que k_element ∈ S_i.
    for k_element in range(n): # k_element representa o índice do elemento (0 a n-1)
        # Encontrar quais subconjuntos cobrem este elemento k_element
        covering_subsets_for_k = []
        for i_subset in range(n): # i_subset representa o índice do subconjunto (0 a n-1)
            if k_element in subsets_covered_elements[i_subset]:
                covering_subsets_for_k.append(x[i_subset])
        
        # Adicionar a restrição de que a soma dos x_i dos subconjuntos que cobrem k_element deve ser >= 1
        model.addConstr(gp.quicksum(covering_subsets_for_k) >= 1, f"cover_element_{k_element+1}")

    # Configurar limite de tempo (10 minutos conforme a Atividade 1, seção 4.2) [8]
    model.Params.TimeLimit = 600 # segundos

    # 8. Otimizar o modelo
    print("Iniciando otimização do MAX-SC-QBF (Modelo Linearizado)...")
    model.optimize()

    # 9. Obter e exibir os resultados
    print("\n" + "=" * 40)
    print("Resultados da Otimização (MAX-SC-QBF Linearizado):")
    print("=" * 40)

    if model.status == GRB.OPTIMAL:
        print(f"Status: Ótimo (Optimal)")
        print(f"Valor da Solução Ótima (Z): {model.ObjVal:.4f}")
        print("Valores das Variáveis x (Subconjuntos Selecionados):")
        selected_subsets = []
        for i in range(n):
            print(f"  x[{i+1}] = {int(x[i].X)}") # Apresentando como x1 a xn
            if x[i].X > 0.5: # Usar uma tolerância para valores binários
                selected_subsets.append(i + 1)
        print(f"Subconjuntos selecionados: {selected_subsets}")
        
    elif model.status == GRB.TIME_LIMIT:
        print(f"Status: Limite de tempo atingido (Time Limit)")
        print(f"Melhor Solução Encontrada (Z): {model.ObjVal:.4f}")
        print(f"Gap de Otimalidade: {model.MIPGap * 100:.2f}%") 
        print("Valores das Variáveis x na melhor solução encontrada:")
        selected_subsets = []
        for i in range(n):
            print(f"  x[{i+1}] = {int(x[i].X)}")
            if x[i].X > 0.5:
                selected_subsets.append(i + 1)
        print(f"Subconjuntos selecionados: {selected_subsets}")
    elif model.status == GRB.INFEASIBLE:
        print("Status: Inviável (Infeasible)")
    else:
        print(f"Status da Otimização: {model.status}")

In [3]:
# geração de instâncias aleatórias

def gerar_subconjuntos(n):
    N = set(range(1, n+1))
    S = []
    
    cobertos = set()
    for i in range(n):
        tamanho = random.randint(2, n)  # tamanho aleatório >=2
        subconjunto = set(random.sample(range(1, n+1), tamanho))
        S.append(subconjunto)
        cobertos.update(subconjunto)
    
    # garante cobertura
    faltando = N - cobertos
    if faltando:
        S[-1] = S[-1].union(faltando)
    
    return S


def gerar_instance(n):
    S = gerar_subconjuntos(n)
    
    linhas = []
    linhas.append(str(n))  # número de elementos
    linhas.append(" ".join(str(len(s)) for s in S))  # tamanhos dos subconjuntos
    
    for s in S:
        linhas.append(" ".join(map(str, sorted(s))))  # elementos de cada subconjunto
    
    # segunda parte -> sequências aleatórias com tamanhos decrescentes
    number = n
    while number >= 1:
        linha = " ".join(str(random.randint(-10, 10)) for _ in range(number))
        linhas.append(linha)
        number -= 1
    
    return "\n".join(linhas)

In [6]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
print("example_instance_content = '''")
print(example_instance_content)
print("'''")

Digite o valor de n:  25


example_instance_content = '''
25
17 9 9 8 3 5 22 22 3 18 24 5 13 15 12 7 13 23 6 8 9 13 20 24 19
3 5 6 7 9 12 13 14 15 16 17 18 19 20 21 22 23
3 8 9 11 15 18 19 20 24
6 8 10 11 16 17 18 20 24
1 3 4 9 15 19 23 24
3 4 16
1 12 14 18 20
1 2 3 4 5 6 7 8 9 10 11 12 13 15 17 18 19 20 21 22 24 25
1 2 3 4 5 6 7 8 9 10 11 13 14 15 16 17 20 21 22 23 24 25
10 18 20
1 3 4 5 6 8 9 10 11 12 16 17 18 19 22 23 24 25
1 2 3 4 5 6 7 8 9 10 11 12 13 15 16 17 18 19 20 21 22 23 24 25
2 3 10 15 19
2 3 4 6 9 12 14 15 17 18 19 20 23
1 3 7 9 12 13 14 15 16 18 19 20 21 23 24
1 3 4 5 7 8 9 13 15 19 22 25
6 12 13 16 18 21 22
1 3 6 8 9 11 13 17 19 20 21 23 25
1 2 3 4 5 6 7 8 9 10 11 13 14 16 17 18 19 20 21 22 23 24 25
4 12 13 19 24 25
4 7 8 9 11 23 24 25
1 3 6 7 10 14 16 20 21
1 4 8 9 11 13 14 15 18 19 20 23 24
2 3 4 5 7 8 9 10 11 12 14 15 16 17 18 19 21 23 24 25
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 21 22 23 24 25
2 3 4 5 6 7 8 10 11 12 13 14 15 16 18 19 20 21 22
-7 -10 -3 -6 8 3 -6 2 -5 5 8 -10 -9 7 -8 

In [7]:
print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content))

--- Executando com n fornecido acima ---
Set parameter TimeLimit to value 600
Iniciando otimização do MAX-SC-QBF (Modelo Linearizado)...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 1000 rows, 650 columns and 2577 nonzeros
Model fingerprint: 0xf0769b65
Variable types: 0 continuous, 650 integer (650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 15.0000000
Presolve removed 536 rows and 334 columns
Presolve time: 0.01s
Presolved: 464 rows, 316 columns, 1303 nonzeros
Variable types: 0 continuous, 316 integer (316 binary)

Root relaxation: objective 3.955000e+02, 238 iterations

In [8]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
print("example_instance_content = '''")
print(example_instance_content)
print("'''")

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content))

Digite o valor de n:  50


example_instance_content = '''
50
45 36 35 3 16 22 7 13 26 11 17 15 50 32 41 21 37 25 15 43 2 14 33 23 18 32 40 12 13 30 30 17 23 50 43 36 39 23 27 7 17 27 12 25 29 15 4 12 18 48
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 27 28 29 30 31 32 33 34 35 37 38 39 41 42 43 44 45 46 47 48 49 50
1 3 5 7 8 12 14 16 18 19 20 21 22 23 25 26 28 29 30 31 32 33 34 35 37 39 40 41 42 43 44 45 46 48 49 50
1 3 4 5 6 8 9 11 12 13 14 15 18 21 22 23 24 26 28 29 30 32 33 34 38 40 41 42 43 45 46 47 48 49 50
16 29 43
6 8 10 11 12 13 15 17 21 25 26 33 42 47 48 49
3 7 8 9 10 11 14 15 21 24 29 35 36 38 41 42 43 44 45 46 49 50
21 24 27 33 39 44 47
1 2 6 10 12 23 28 29 30 33 36 39 49
1 3 5 9 12 14 15 18 19 20 21 29 33 34 35 36 38 39 40 41 43 44 47 48 49 50
2 3 6 16 19 20 28 29 32 35 44
1 2 3 5 7 11 12 13 17 21 26 30 38 39 41 48 49
4 6 11 14 18 19 21 22 24 25 28 38 41 46 47
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 4

In [9]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
print("example_instance_content = '''")
print(example_instance_content)
print("'''")

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content))

Digite o valor de n:  100


example_instance_content = '''
100
78 78 17 46 37 45 45 2 80 75 44 79 75 49 29 45 99 97 12 22 100 18 16 29 8 46 66 15 23 51 53 66 90 7 74 77 88 69 59 18 60 72 87 73 92 22 71 84 58 75 32 64 43 84 15 54 71 23 48 78 85 90 22 23 28 2 29 16 20 97 30 3 86 33 66 60 41 64 27 52 84 77 62 77 11 14 14 41 94 67 53 65 26 24 11 70 28 17 45 44
1 2 3 4 6 7 8 9 10 11 12 13 14 15 16 17 18 19 21 22 23 25 26 27 28 29 31 33 34 36 37 39 43 44 45 46 47 48 49 50 54 56 57 58 59 61 62 63 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 83 84 85 86 88 90 91 92 93 95 96 97 99 100
2 3 5 7 8 9 10 11 12 13 15 16 17 18 19 20 21 22 26 28 29 30 31 32 34 35 36 38 41 42 45 46 47 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 72 73 74 75 77 78 79 80 81 82 84 85 86 87 88 89 91 92 93 94 96 97 98 99
2 12 17 33 37 43 47 51 54 55 59 61 66 74 77 84 85
3 4 8 9 12 13 14 16 18 21 24 26 28 29 31 37 38 40 43 47 50 51 52 53 55 57 59 60 62 63 65 66 70 73 74 75 76 80 81 82 87 89 91 94 97 99
1 3 5 9 10 11 16 17 18 20 22 2

In [10]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
print("example_instance_content = '''")
print(example_instance_content)
print("'''")

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content))

Digite o valor de n:  200


example_instance_content = '''
200
174 74 105 85 55 119 29 95 18 124 148 71 28 74 42 24 95 154 17 58 116 54 122 64 112 43 34 30 120 95 91 46 74 61 152 33 13 65 58 186 19 32 37 15 166 35 154 88 119 104 82 160 194 78 178 166 144 200 140 48 39 88 45 139 79 173 64 51 33 171 107 187 110 176 136 48 143 101 14 186 130 172 27 11 18 35 162 36 161 48 17 50 142 25 16 133 122 116 6 68 128 26 189 42 72 60 101 26 162 74 39 97 73 139 65 186 37 196 53 118 87 193 70 109 199 12 195 165 170 68 83 113 96 34 43 93 4 177 16 65 134 67 4 38 118 10 72 45 124 27 22 77 58 126 29 55 130 148 198 153 95 111 60 8 59 158 128 57 41 138 76 101 19 14 84 64 129 141 118 68 150 51 140 129 199 135 71 147 172 52 51 196 132 128 123 13 199 166 90 94
1 3 4 5 6 7 9 10 11 12 13 15 16 17 18 19 20 22 23 24 26 27 28 29 30 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 60 61 62 63 64 65 66 67 68 69 70 71 72 73 77 78 79 80 81 82 83 84 85 87 89 90 91 92 93 94 95 96 98 100 101 102 103 104 105 106 107 109 1

In [11]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
print("example_instance_content = '''")
print(example_instance_content)
print("'''")

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content))

Digite o valor de n:  400


example_instance_content = '''
400
147 233 56 269 143 395 375 8 196 111 277 151 268 349 373 395 81 11 321 362 332 165 173 352 229 98 287 214 261 77 132 273 393 149 353 193 317 26 224 145 382 375 153 200 8 206 19 228 334 174 320 297 137 335 29 2 150 224 99 221 89 319 277 3 81 382 292 228 5 271 381 16 394 308 132 198 242 92 18 277 291 358 43 122 275 150 219 151 315 326 186 333 322 25 48 49 330 384 78 66 237 246 385 282 180 400 106 15 187 340 208 11 130 225 253 44 264 333 31 309 288 226 362 197 296 122 242 63 308 26 364 68 321 290 322 365 373 187 268 259 183 65 266 307 245 149 281 23 122 88 39 319 203 183 310 111 256 359 287 387 42 55 262 377 169 121 120 25 243 337 83 31 388 169 342 93 121 308 373 181 294 16 192 195 20 13 335 257 61 57 321 110 67 364 223 215 106 53 60 279 156 387 14 278 339 222 249 113 43 365 79 19 15 105 390 293 262 193 157 257 170 245 5 99 209 119 8 103 140 361 234 381 121 137 30 229 86 236 6 195 232 382 374 344 192 170 148 120 215 145 178 75 83 237 264 280 245 270 352 